In [1]:
from bs4 import BeautifulSoup
import time 
import pandas
import requests

In [2]:
def load_html(url):
    response = requests.get(url)
    if not response.ok:
        print(f"{response.status_code}, url: {url}")
    return response.text

In [3]:
url = "http://regionalip.aripo.org/wopublish-search/public/home?1"

In [4]:
pages = load_html(url)

In [5]:
soup = BeautifulSoup(pages, 'lxml')
target_page = soup.find('div', class_='col-md-5 landing-page-table-container')
rows = target_page.find_all_next('div', class_='row')
del(rows[0])

In [6]:
flags, countries, patents, designs, trademarks = list(), list(), list(), list(), list()

In [7]:
class Data():

    def __init__(self, flag, country, patent, design, trademark):

        self.flag = flag
        self.country = country
        self.patent = patent
        self.design = design
        self.trademark = trademark

    def describe(self):
        print(f"\
            flag : {self.flag} \n\
            country : {self.country} \n\
            patent : {self.patent}\n\
            design : {self.design} \n\
            trademark : {self.trademark}\n\
        ")


all_datas = list()

In [8]:
for i in range(len(rows)):
    flag = rows[i].find('img', class_='primary-border-color')['src']
    country = rows[i].find('div', class_='geo-grid-drawing-container col-md-2 countryBodyClass').text
    patent = rows[i].find('div', class_='geo-grid-data col-md-2 patentBodyClass').text
    try:
        design = rows[i].find('div', class_='geo-grid-data col-md-2 designBodyClass').find('div', class_='countButtonClass primary-btn-color').text
    except:
        design = rows[i].find('div', class_='geo-grid-data col-md-2 designBodyClass').find('div', class_='countButtonClass-zero').text
    try:
        trademark = rows[i].find('div', class_='geo-grid-data col-md-2 tmBodyClass').find('div', class_='countButtonClass primary-btn-color').text 
    except:
        rows[i].find('div', class_='geo-grid-data col-md-2 tmBodyClass').find('div', class_='countButtonClass-zero').text
    
    new_data = Data(flag=flag, country=country, design=design, patent=patent, trademark=trademark)
    all_datas.append(new_data)

In [9]:
for data in all_datas:
    flags.append(data.flag)
    countries.append(data.country)
    patents.append(data.patent)
    designs.append(data.design)
    trademarks.append(data.trademark)


In [10]:
table = pandas.DataFrame({"Flags": flags, "Countries": countries, "Patents": patents, "Designs": designs, "Trademarks":trademarks})

In [11]:
table.to_excel('regionalip.xlsx')